In [1]:
import django
from django.conf import settings
import dotenv
import pandas
import pathlib
import os

In [2]:
BASE_DIR = pathlib.Path('.').parent.absolute()
BASE_DIR

WindowsPath('d:/coding/websites/mydatabase/djangobackend')

In [3]:
dotenv.load_dotenv(BASE_DIR / '.env')

True

In [4]:
REDIS_HOST = os.getenv('REDIS_HOST', '127.0.0.1')
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')
REDIS_URL = f'redis://:{REDIS_PASSWORD}@{REDIS_HOST}:6379'

RABBITMQ_HOST = os.getenv('RABBITMQ_HOST', 'localhost')
RABBITMQ_USER = os.getenv('RABBITMQ_DEFAULT_USER')
RABBITMQ_PASSWORD = os.getenv('RABBITMQ_DEFAULT_PASS')

In [5]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'true')

'true'

In [6]:
try:
    settings.configure(**{
        'BASE_DIR': BASE_DIR,
        'DEBUG': True,
        'INSTALLED_APPS': [
            'tabledocuments',
            'dbschemas',
            'dbtables',
            'endpoints'
        ],
        'SECRET_KEY': os.getenv('SECRET_KEY'),
        'DATABASES': {
            'default': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': BASE_DIR / 'db.sqlite3',
            }
        },
        'MEDIA_URL': 'media/',
        'MEDIA_ROOT': BASE_DIR / 'media',
        'REDIS_URL': f'redis://:{REDIS_PASSWORD}@{REDIS_HOST}:6379',
        'CELERY_BROKER_URL': f'amqp://{RABBITMQ_USER}:{RABBITMQ_PASSWORD}@{RABBITMQ_HOST}:5672',
        'CELERY_RESULT_BACKEND': REDIS_URL,
        'CELERY_ACCEPT_CONTENT': ['json'],
        'CELERY_TASK_SERIALIZER': 'json',
        'CELERY_RESULT_SERIALIZER': 'json',
        'CELERY_TIMEZONE': 'Europe/Oslo',
        'CELERY_BEAT_SCHEDULER': 'django_celery_beat.schedulers:DatabaseScheduler',
        'CELERY_BROKER_CONNECTION_RETRY_ON_STARTUP': True,
        'CHANNEL_LAYERS': {
            'default': {
                'BACKEND': 'channels_redis.core.RedisChannelLayer',
                'CONFIG': {
                    'hosts': [REDIS_URL]
                }
            }
        }
    })
except:
    pass

In [7]:
try:
    django.setup()
except:
    pass

In [8]:
from dbtables.models import DatabaseTable
from tabledocuments import tasks

In [9]:
table = DatabaseTable.objects.first()
document = table.documents.first()
document

<TableDocument: Google  document>

In [10]:
path = os.getenv('TEST_CSV_FILE')

In [11]:
column_options = [
    {
        "name": "first_name",
        "new_name": "firstname",
        "columnType": "String",
        "unique": True,
        "nullable": False,
        "visible": True
    },
    {
        "name": "last-name",
        "new_name": "lastname",
        "columnType": "String",
        "unique": False,
        "nullable": True,
        "visible": True
    },
    {
        "name": "age",
        "new_name": "age",
        "columnType": "String",
        "unique": False,
        "nullable": True,
        "visible": True
    },
    {
        "name": "date_Of_birth",
        "new_name": "date_of_birth",
        "columnType": "String",
        "unique": False,
        "nullable": True,
        "visible": True
    },
    {
        "name": "talent",
        "new_name": "talent",
        "columnType": "Boolean",
        "unique": False,
        "nullable": False,
        "visible": True
    }
]

In [12]:
with open(path, mode='rb') as f:
    tasks.create_csv_file_from_data.apply_async(
        args=[
            f.read(), 
            document.id, 
            None, 
            column_options
        ]
    )